In [10]:
import inro.modeller as m
import os
import glob
mm = m.Modeller()
eb = mm.emmebank
import inro.emme.matrix as _matrix
import numpy as np

matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
change_matrix = m.Modeller().tool("inro.emme.data.matrix.change_matrix_properties") 
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")  
matrix_transaction = mm.tool('inro.emme.data.matrix.matrix_transaction')

def import_matrices_from_directory(matrix_folder, matrix_list, extension, scenario):
    #imports all matrices in directory and subfolders of the specified extension type
    #matrix_list should be in the format:
    #   {file_name1: matrix_num1, file_name2: matrix_num2, ...}
    import os
    
    mat_imported = []
    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(extension):
                name = os.path.splitext(matrix_file)[0]
                if name in matrix_list:
                    matrix_number = int(matrix_list[name][2:])
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_") )
                    print "Imported matrix " + matrix_list[name]
                    change_matrix(matrix = matrix_list[name],
                        matrix_name = name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_")[:40],
                        matrix_description = name)
                    mat_imported.append(matrix_list[name])
    
    #check
    for name in matrix_list:
        if eb.matrix(matrix_list[name]): 
            n_matrix = eb.matrix(matrix_list[name]).get_numpy_data(scenario)
            if n_matrix.sum() == 0 :
                print "Matrix %s was not imported, or contains no values!" % name
        else:
            print "Matrix %s was not imported!" % name
    print "Finished Importing Matrices"
    return mat_imported

In [8]:
### Folder Location
folder = r"C:\Users\pechen\Desktop\BoltonGO_TripsAnalysis"

### Matrices to Import from the folder
transit_ztsa = "trips_peak_transit_ztsa"
mat_list = {transit_ztsa:""}

for matrix_name in mat_list.keys():
    mat_list[matrix_name] = eb.available_matrix_identifier('FULL')
    create_matrix(matrix_id = mat_list[matrix_name],
              matrix_name= matrix_name.replace(" ","_"),
              matrix_description = matrix_name)

### function(folder_name, matrix_list, format, scen)
import_matrices_from_directory(folder, mat_list, '.mdf', 9999)

Imported matrix mf24
Finished Importing Matrices


In [41]:
### Folder Location
folder = r"C:\Users\pechen\Desktop\BoltonGO_TripsAnalysis\2051Pref_BoltonGO"

### Matrices to Import from the folder
default_name = 'trips_am_peak_'
auto1_nt = default_name + 'auto1_nt'
auto1_toll = default_name + 'auto1_toll'
auto2_nt_hov = default_name + 'auto2_nt_hov'
auto2_nt_nh = default_name + 'auto2_nt_nh'
auto2_toll_hov = default_name + 'auto2_toll_hov'
auto2_toll_nh = default_name + 'auto2_toll_nh'
auto3_nt_hov = default_name + 'auto3_nt_hov'
auto3_nt_hh = default_name + 'auto3_nt_nh'
auto3_toll_hov = default_name + 'auto3_toll_hov'
auto3_toll_nh = default_name + 'auto3_toll_nh'
mat_list = {auto1_nt:"", 
            auto1_toll:"",
            auto2_nt_hov:"",
            auto2_nt_nh:"",
            auto2_toll_hov:"",
            auto2_toll_nh:"",
            auto3_nt_hov:"",
            auto3_nt_hh:"",
            auto3_toll_hov:"",
            auto3_toll_nh:"",
           }

for matrix_name in mat_list.keys():
    mat_list[matrix_name] = eb.available_matrix_identifier('FULL')
    create_matrix(matrix_id = mat_list[matrix_name],
              matrix_name= matrix_name.replace(" ","_"),
              matrix_description = matrix_name)

### function(folder_name, matrix_list, format, scen)
import_matrices_from_directory(folder, mat_list, '.mdf', 710)

Imported matrix mf74
Imported matrix mf78
Imported matrix mf71
Imported matrix mf77
Imported matrix mf73
Imported matrix mf76
Imported matrix mf79
Imported matrix mf72
Imported matrix mf75
Imported matrix mf70
Finished Importing Matrices


In [42]:
import pandas as pd
# matrix to pandas 
def mat_to_pddf(id, scen):
    mat = eb.matrix(id).get_data(scen)
    zones_tupl = mat.indices
    indices = pd.Index(zones_tupl[0])
    indices.name = 'p'
    cols = pd.Index(zones_tupl[1])
    cols.name = 'q'
    matrix = mat.to_numpy()
    return pd.DataFrame(matrix, index=indices, columns=cols)

In [43]:
mats = ['mf74', 'mf78', 'mf71', 'mf77', 'mf73', 'mf76', 'mf79', 'mf72', 'mf75', 'mf70']

total_auto_am = 0
for i in mats:
    i = mat_to_pddf(i, 9999)
    total_auto_am += i

In [44]:
total_auto_am.head()

q,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,9922,9923,9924,9925,9926,9927,9928,9983,9984,9985
p,,,,,,,,,,,,,,,,,,,,,
1001,35.572468,13.453135,21.967438,12.494029,9.747603,20.961950,1.116435,4.454873,7.535470,10.579278,...,0.0,0.0,1.716674e-05,0.0,0.0,0.0,0.245293,0.0,0.000049,2.724921e-08
1002,17.302555,28.450274,27.477701,22.260183,13.449595,11.251068,2.304911,5.335682,6.697052,14.312964,...,0.0,0.0,9.140953e-06,0.0,0.0,0.0,0.195655,0.0,0.000000,9.988133e-10
1003,9.190888,14.136268,48.328323,23.923407,8.423223,7.708669,9.664411,18.254787,4.197792,21.504427,...,0.0,0.0,7.745597e-07,0.0,0.0,0.0,0.107912,0.0,0.000048,2.025407e-08
1004,5.621179,10.021130,22.071672,16.393066,4.710692,4.164278,2.743141,5.961917,3.896481,9.883012,...,0.0,0.0,1.231804e-02,0.0,0.0,0.0,0.027138,0.0,0.000022,2.124721e-10
1005,2.164529,3.730655,2.690474,1.280841,2.296010,3.407453,0.203820,1.159595,1.130306,4.165790,...,0.0,0.0,2.930021e-06,0.0,0.0,0.0,0.001124,0.0,0.000000,1.464470e-07


In [45]:
total_auto_am.head()
loc = r"C:\Users\pechen\Desktop\BoltonGO_TripsAnalysis"
zone = "BoltonZones.csv"
zonekeep = pd.read_csv(os.path.join(loc, zone))
bolton_auto = zonekeep.merge(total_auto_am, left_on = "TAZ", right_on="p", how="inner").set_index('TAZ')
bolton_auto.loc['total_caledon'] = bolton_auto.sum()
bolton_auto = bolton_auto.loc['total_caledon'].transpose()
bolton_auto.to_csv(os.path.join(loc, "bolton_auto_summary.csv"), header=True)

### Transit Only 

In [56]:
### Folder Location
folder = r"C:\Users\pechen\Desktop\BoltonGO_TripsAnalysis\2051Pref\Matrices"

### Matrices to Import from the folder
transit_demand = "TRIPS TOTAL TRANSIT peak"
mat_list = {transit_demand:""}

for matrix_name in mat_list.keys():
    mat_list[matrix_name] = eb.available_matrix_identifier('FULL')
    create_matrix(matrix_id = mat_list[matrix_name],
              matrix_name= matrix_name.replace(" ","_"),
              matrix_description = matrix_name)

### function(folder_name, matrix_list, format, scen)
import_matrices_from_directory(folder, mat_list, '.mdf', 701)

Imported matrix mf86
Finished Importing Matrices


In [57]:
boltons_transdem = mat_to_pddf('mf86', 701)

loc = r"C:\Users\pechen\Desktop\BoltonGO_TripsAnalysis"
zone = "BoltonZones.csv"
zonekeep = pd.read_csv(os.path.join(loc, zone))
boltons_transdem = zonekeep.merge(boltons_transdem, left_on = "TAZ", right_on="p", how="inner").set_index('TAZ')
boltons_transdem.loc['total_caledon'] = boltons_transdem.sum()
boltons_transdem = boltons_transdem.loc['total_caledon'].transpose()
boltons_transdem.to_csv(os.path.join(loc, "bolton_transdem_summary.csv"), header=True)

## Auto

In [58]:
### Folder Location
folder = r"C:\Users\pechen\Desktop\BoltonGO_TripsAnalysis\2051Pref\Matrices"

### Matrices to Import from the folder
transit_demand = "TRIPS TOTAL AUTO peak"
mat_list = {transit_demand:""}

for matrix_name in mat_list.keys():
    mat_list[matrix_name] = eb.available_matrix_identifier('FULL')
    create_matrix(matrix_id = mat_list[matrix_name],
              matrix_name= matrix_name.replace(" ","_"),
              matrix_description = matrix_name)

### function(folder_name, matrix_list, format, scen)
import_matrices_from_directory(folder, mat_list, '.mdf', 701)

Imported matrix mf87
Finished Importing Matrices


In [59]:
boltons_transdem = mat_to_pddf('mf87', 701)

loc = r"C:\Users\pechen\Desktop\BoltonGO_TripsAnalysis"
zone = "BoltonZones.csv"
zonekeep = pd.read_csv(os.path.join(loc, zone))
boltons_transdem = zonekeep.merge(boltons_transdem, left_on = "TAZ", right_on="p", how="inner").set_index('TAZ')
boltons_transdem.loc['total_caledon'] = boltons_transdem.sum()
boltons_transdem = boltons_transdem.loc['total_caledon'].transpose()
boltons_transdem.to_csv(os.path.join(loc, "boltons_peakauto_summary.csv"), header=True)